In [18]:
# -=-<[ Bismillahirrahmanirrahim ]>-=-
# -*- coding: utf-8 -*-
# @Date    : 2024-10-15 17:07:17
# @Author  : Dahir Muhammad Dahir (dahirmuhammad3@gmail.com)
# @Link    : link
# @Version : 1.0.0

from utils.crud_util import CrudUtil
from config.database import db_session_manager as db
import models
import schemas





In [4]:
from mixins.schemas import PagedResult


async def create_faculty(name: str) -> models.Faculty:
    async with db.session() as session:
        cu: CrudUtil = CrudUtil(session)

        faculty = schemas.FacultyIn(name=name)
        db_faculty: models.Faculty = await cu.create_model(models.Faculty, faculty)
        return db_faculty


async def create_department(name: str, faculty_id: str) -> models.Department:
    async with db.session() as session:
        cu: CrudUtil = CrudUtil(session)

        department = schemas.DepartmentIn(name=name, faculty_id=faculty_id)
        db_department: models.Department = await cu.create_model(models.Department, department)
        return db_department


async def get_faculty_by_id(faculty_id: str) -> models.Faculty | None:
    async with db.session() as session:
        cu: CrudUtil = CrudUtil(session)
        db_faculty = await cu.get_model_or_none(models.Faculty, {"faculty_id": faculty_id})

        return db_faculty

async def get_department_by_id(department_id: str) -> models.Department | None:
    async with db.session() as session:
        cu: CrudUtil = CrudUtil(session)
        db_department = await cu.get_model_or_none(models.Department, {"department_id": department_id})

        return db_department


async def get_all_faculties() -> list[models.Faculty]:
    async with db.session() as session:
        cu: CrudUtil = CrudUtil(session)
        db_faculties: PagedResult[models.Faculty] = await cu.list_model(models.Faculty)
        return db_faculties.data


async def get_all_departments() -> list[models.Department]:
    async with db.session() as session:
        cu: CrudUtil = CrudUtil(session)
        db_departments: PagedResult[models.Department] = await cu.list_model(models.Department)
        return db_departments.data


In [9]:
data = {
    "faculties": [
        {"name": "Faculty of Science", "departments": [
            {"name": "Department of Physics"},
            {"name": "Department of Chemistry"},
            {"name": "Department of Biology"},]},
        {"name": "Faculty of Arts", "departments": [
            {"name": "Department of Literature"},
            {"name": "Department of History"},]},
        {"name": "Faculty of Engineering", "departments": [
            {"name": "Department of Computer Science"},
            {"name": "Department of Electrical Engineering"},]},
        {"name": "Faculty of Business", "departments": [
            {"name": "Department of Marketing"},
            {"name": "Department of Finance"},]},
    ]
}

In [11]:

async def seed_data():
    for faculty_data in data["faculties"]:
        faculty_name = faculty_data["name"]
        db_faculty = await create_faculty(faculty_name)
        print(f"Created Faculty: {db_faculty}")

        for department_data in faculty_data["departments"]:
            department_name = department_data["name"]
            db_department = await create_department(department_name, db_faculty.uuid)
            print(f"  Created Department: {db_department} under Faculty: {db_faculty.name}")

In [12]:
await seed_data()

2025-10-16 02:16:48,403 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-10-16 02:16:48,403 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-16 02:16:48,405 INFO sqlalchemy.engine.Engine select current_schema()
2025-10-16 02:16:48,405 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-16 02:16:48,406 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-10-16 02:16:48,406 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-16 02:16:48,406 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-16 02:16:48,408 INFO sqlalchemy.engine.Engine INSERT INTO faculties (name, display_name, uuid, date) VALUES ($1::VARCHAR, $2::VARCHAR, $3::VARCHAR, $4::DATE) RETURNING faculties.created_at, faculties.last_modified
2025-10-16 02:16:48,408 INFO sqlalchemy.engine.Engine [generated in 0.00029s] ('FACULTY OF SCIENCE', 'FACULTY OF SCIENCE', '0199ea974d9875e29325cdf3790105dd', datetime.date(2025, 10, 16))
2025-10-16 02:16:48,409 INFO sqlalchemy.engine.Engine COMMIT
2025-10-16

In [13]:
faculties = await get_all_faculties()

2025-10-16 02:19:53,722 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-16 02:19:53,723 INFO sqlalchemy.engine.Engine SELECT count(faculties.uuid) AS count_1 
FROM faculties 
WHERE faculties.uuid IS NOT NULL
2025-10-16 02:19:53,723 INFO sqlalchemy.engine.Engine [generated in 0.00031s] ()
2025-10-16 02:19:53,727 INFO sqlalchemy.engine.Engine SELECT faculties.name, faculties.display_name, faculties.uuid, faculties.date, faculties.created_at, faculties.last_modified 
FROM faculties ORDER BY faculties.uuid ASC 
 LIMIT $1::INTEGER OFFSET $2::INTEGER
2025-10-16 02:19:53,727 INFO sqlalchemy.engine.Engine [generated in 0.00060s] (100, 0)
2025-10-16 02:19:53,730 INFO sqlalchemy.engine.Engine SELECT departments.faculty_id AS departments_faculty_id, departments.name AS departments_name, departments.display_name AS departments_display_name, departments.uuid AS departments_uuid, departments.date AS departments_date, departments.created_at AS departments_created_at, departments.last_modified 

In [22]:
faculty_schemas = [schemas.FacultyOut.model_validate(faculty) for faculty in faculties]
# for fs in faculty_schemas:
#     print(fs)

for faculty in faculties:
    for department in faculty.departments:
        print(f"Faculty: {faculty.name}, Department: {department.name}")
    print("-----")

Faculty: FACULTY OF SCIENCE, Department: DEPARTMENT OF BIOLOGY
Faculty: FACULTY OF SCIENCE, Department: DEPARTMENT OF CHEMISTRY
Faculty: FACULTY OF SCIENCE, Department: DEPARTMENT OF PHYSICS
-----
Faculty: FACULTY OF ARTS, Department: DEPARTMENT OF HISTORY
Faculty: FACULTY OF ARTS, Department: DEPARTMENT OF LITERATURE
-----
Faculty: FACULTY OF ENGINEERING, Department: DEPARTMENT OF ELECTRICAL ENGINEERING
Faculty: FACULTY OF ENGINEERING, Department: DEPARTMENT OF COMPUTER SCIENCE
-----
Faculty: FACULTY OF BUSINESS, Department: DEPARTMENT OF FINANCE
Faculty: FACULTY OF BUSINESS, Department: DEPARTMENT OF MARKETING
-----


In [23]:
departments = await get_all_departments()

2025-10-16 02:30:59,697 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-16 02:30:59,699 INFO sqlalchemy.engine.Engine SELECT count(departments.uuid) AS count_1 
FROM departments 
WHERE departments.uuid IS NOT NULL
2025-10-16 02:30:59,699 INFO sqlalchemy.engine.Engine [generated in 0.00047s] ()
2025-10-16 02:30:59,702 INFO sqlalchemy.engine.Engine SELECT departments.name, departments.faculty_id, departments.display_name, departments.uuid, departments.date, departments.created_at, departments.last_modified, faculties_1.name AS name_1, faculties_1.display_name AS display_name_1, faculties_1.uuid AS uuid_1, faculties_1.date AS date_1, faculties_1.created_at AS created_at_1, faculties_1.last_modified AS last_modified_1 
FROM departments LEFT OUTER JOIN faculties AS faculties_1 ON faculties_1.uuid = departments.faculty_id ORDER BY departments.uuid ASC 
 LIMIT $1::INTEGER OFFSET $2::INTEGER
2025-10-16 02:30:59,702 INFO sqlalchemy.engine.Engine [generated in 0.00048s] (100, 0)
2025-10-1

In [25]:
departments_schemas = [schemas.DepartmentOut.model_validate(dept) for dept in departments]
# for ds in departments_schemas:
#     print(ds)

for dept in departments:
    print(f"Department: {dept.name}, Faculty ID: {dept.faculty_id}, Faculty Name: {dept.faculty.name}")

Department: DEPARTMENT OF PHYSICS, Faculty ID: 0199ea974d9875e29325cdf3790105dd, Faculty Name: FACULTY OF SCIENCE
Department: DEPARTMENT OF CHEMISTRY, Faculty ID: 0199ea974d9875e29325cdf3790105dd, Faculty Name: FACULTY OF SCIENCE
Department: DEPARTMENT OF BIOLOGY, Faculty ID: 0199ea974d9875e29325cdf3790105dd, Faculty Name: FACULTY OF SCIENCE
Department: DEPARTMENT OF LITERATURE, Faculty ID: 0199ea974dc27cf397ba1b885ded61e1, Faculty Name: FACULTY OF ARTS
Department: DEPARTMENT OF HISTORY, Faculty ID: 0199ea974dc27cf397ba1b885ded61e1, Faculty Name: FACULTY OF ARTS
Department: DEPARTMENT OF COMPUTER SCIENCE, Faculty ID: 0199ea974dd779f1b50529b043e81db4, Faculty Name: FACULTY OF ENGINEERING
Department: DEPARTMENT OF ELECTRICAL ENGINEERING, Faculty ID: 0199ea974dd779f1b50529b043e81db4, Faculty Name: FACULTY OF ENGINEERING
Department: DEPARTMENT OF MARKETING, Faculty ID: 0199ea974deb79c0adaf4a070c09c510, Faculty Name: FACULTY OF BUSINESS
Department: DEPARTMENT OF FINANCE, Faculty ID: 0199ea9